In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy as sp

from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [ ]:
data = pd.read_csv("~/datacsv/HousingData.csv")

data.isnull().sum()

CRIM       20
ZN         20
INDUS      20
CHAS       20
NOX         0
RM          0
AGE        20
DIS         0
RAD         0
TAX         0
PTRATIO     0
B           0
LSTAT      20
MEDV        0
dtype: int64

In [ ]:
data=data.dropna()

In [ ]:
data.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [ ]:
x = data.iloc[:,0:13].values
y=data.iloc[:,13].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split( x,y , test_size= 0.20)

In [ ]:
#(train_data, train_targets), (test_data, test_targets)

In [ ]:
print(f'Training data : {x_train.shape}')
print(f'Test data : {x_test.shape}')
print(f'Training sample : {y_train[0]}')
print(f'Training target sample : {y_test[0]}')

Training data : (315, 13)
Test data : (79, 13)
Training sample : 17.8
Training target sample : 33.0


In [ ]:
train_data = x_train
test_data = x_test
train_targets = y_train
test_targets = y_test

In [ ]:
print(f'Training data : {train_data.shape}')
print(f'Test data : {test_data.shape}')
print(f'Training sample : {train_data[0]}')
print(f'Training target sample : {train_targets[0]}')

Training data : (315, 13)
Test data : (79, 13)
Training sample : [  0.54452   0.       21.89      0.        0.624     6.151    97.9
   1.6687    4.      437.       21.2     396.9      18.46   ]
Training target sample : 17.8


In [ ]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [ ]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))

    model.compile(optimizer='adam',      #optimizer='rmsprop'
              loss='mse',
              metrics=['mae'])
    return model

In [ ]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print(f'Processing fold # {i}')
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
                            [train_data[:i * num_val_samples],
                            train_data[(i+1) * num_val_samples:]],
                            axis=0)
    partial_train_targets = np.concatenate(
                            [train_targets[:i * num_val_samples],
                            train_targets[(i+1)*num_val_samples:]],
                            axis=0)
    model = build_model()
    model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=1,
              verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

Processing fold # 0
Processing fold # 1
Processing fold # 2
Processing fold # 3


In [ ]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')

all_scores : [2.1242644786834717, 2.338365077972412, 2.4996912479400635, 2.678586483001709]
mean all scores : 2.410226821899414


In [ ]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

3/3 [==============================] - 0s 1ms/step - loss: 9.8062 - mae: 2.3911


In [ ]:
test_mae_score

2.39113187789917

In [ ]:
# test_mae_score with optimizer='rmsprop' is 2.57

# test_mae_score with optimizer='adam' is 2.39